In [ ]:
import os
from midi2img import midi2image
from img2midi import image2midi
import glob
import numpy as np
import tensorflow as tf
from PIL import Image
from skimage import img_as_ubyte
import matplotlib.pyplot as plt

In [ ]:
## convert midi to image
isExist = os.path.exists("../res/midi2img")
if not isExist:
    os.makedirs("../res/midi2img")

os.chdir("../res/midi2img")
midi_file = glob.glob("../../data/mozart/*.mid")

for midi in midi_file:
    try:
        midi = midi2image(midi)
    except:
        pass

os.chdir("..")

In [ ]:
## preprocess the images

def input_prep_fn(x):
    
    out = tf.image.resize(x, size = [106,106]) / 255.0
    out = np.array(out)
    out[out > 0] = 1
    return 2 * out - 1

image_file = glob.glob("midi2img/*.png")
image_file = image_file
images = []
for i in image_file:
    image = tf.keras.utils.load_img(i, color_mode = "grayscale")
    image = tf.keras.utils.img_to_array(image)
    images.append(image) 
images = np.array(images)
images = input_prep_fn(images)
print(images.shape)

In [ ]:
np.unique(images)

In [ ]:
## check the images
plt.figure(figsize=(10, 10))
for i in range(9):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(np.array(images[i]), cmap='gray')
  plt.axis("off")

In [ ]:
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten,BatchNormalization
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout

def define_discriminator(in_shape = (106,106,1)):
    model = Sequential()
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.5))
    model.add(Conv2D(128, (3,3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid'))
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

def define_generator(latent_dim):
    model = Sequential()
    n_nodes = 256 * 53 * 53
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(BatchNormalization()) ##
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((53, 53, 256)))
    model.add(Conv2DTranspose(128, (4,4), strides=(1,1), padding='same', use_bias=False)) #
    model.add(LeakyReLU(alpha=0.2)) #
    model.add(Conv2DTranspose(64, (4,4), strides=(2,2), padding='same', use_bias=False))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(1, (7,7) , padding='same',activation = 'tanh', use_bias=False))
    return model

def define_gan(g_model, d_model):
    d_model.trainable = False
    model = Sequential()
    model.add(g_model)
    model.add(d_model)
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

def generate_real_samples(dataset, n_samples):
    ix = randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]
    y = ones((n_samples, 1))
    return X, y
 
def generate_latent_points(latent_dim, n_samples):
    x_input = randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input
def generate_fake_samples(g_model, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim, n_samples)
    X = g_model.predict(x_input, verbose=0)
    y = zeros((n_samples, 1))
    return X, y

def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=51, n_batch=10, file_dir = "model_test"):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)

    d_loss_list = []
    g_loss_list = [] 
    for i in range(n_epochs):
        print("Epoch %d" %(i+1))
        for j in range(bat_per_epo):
            
            X_real, y_real = generate_real_samples(dataset, half_batch)
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))

            #d_model.trainable = True
            d_loss, _ = d_model.train_on_batch(X, y)

            X_gan = generate_latent_points(latent_dim, n_batch)
            y_gan = ones((n_batch, 1))

            #d_model.trainable = False
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            print('>%d/%d, d=%.3f, g=%.3f' % (j+1, bat_per_epo, d_loss, g_loss))

        if (i+1) % 10 == 0:
            summarize_performance(file_dir, i, g_model, d_model, dataset, latent_dim)
            #clear_output()
            
        d_loss_list.append(d_loss)
        g_loss_list.append(g_loss)
    
    return d_loss_list, g_loss_list

def summarize_performance(file_dir, epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
    X_real, y_real = generate_real_samples(dataset, n_samples)
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    filename = 'generator_model_%03d.h5' % (epoch + 1)

    isExist = os.path.exists(file_dir)
    if not isExist:
        os.makedirs(file_dir)
    g_model.save(file_dir + "/" + filename)

In [ ]:
## initialize the model

latent_dim = 100
d_model = define_discriminator()
g_model = define_generator(latent_dim)
gan_model = define_gan(g_model, d_model)
print(images.shape)
print(np.unique(images))

In [ ]:
d_loss, g_loss = train(g_model, d_model, gan_model, images, latent_dim, n_epochs = 400, n_batch=50, file_dir = "model_out")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3))
ax1.plot(d_loss, c = 'b')
ax1.set_title("Discriminator Loss")

ax2.plot(g_loss, c = 'r')
ax2.set_title("Generator Loss")
fig.supxlabel('Epoch')
plt.show()
fig.savefig("res/model_out/" + 'loss.png')

In [ ]:
import keras

latent_dim = 100
sim_input = generate_latent_points(latent_dim, 1)

In [ ]:
model_index = ["010", "020", "030", "040", "050", "060", "070", "080", "090"]
#fig, axs = plt.subplots(len(model_index),figsize=(20, 3.5))
for index, value in enumerate(model_index):
    model = keras.models.load_model('res/model_out/generator_model_' + value +'.h5', compile=False)
    X = model.predict(sim_input, verbose=0)
    X = np.squeeze(X)
    plt.subplot(3,3,index+1)
    #plt.subplots(1)
    plt.title("Epoch "+value)
    plt.axis('off')
    plt.imshow(X, cmap='gray')
    plt.savefig("res/model_out/" + 'train_examine.png')

In [ ]:
## generate music
model = keras.models.load_model('res/model_out/generator_model_400.h5', compile=False)

latent_dim = 100
latent_points = generate_latent_points(latent_dim, 1)

out = model.predict(latent_points)
out = np.squeeze(out)

img = img_as_ubyte(out)
plt.imshow(img, cmap='gray')
new_image = Image.fromarray(img,'L')
new_image = new_image.save('res/model_out/composition.png')

image2midi("composition.png")

### Post process stage to Jazzify the output